In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 2**22)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [3]:
pi_str = """3.14159 26535 89793 23846 26433 83279 50288 41971 69399 37510 
58209 74944 59230 78164 06286 20899 86280 34825 34211 70679 82148 08651 
32823 06647 09384 46095 50582 23172 53594 08128 48111 74502 84102 70193 
85211 05559 64462 29489 54930 38196 44288 10975 66593 34461 28475 64823 
37867 83165 27120 19091 45648"""
pi_str = pi_str.replace(' ', '').replace('.', '').replace('\n', '')
pi = np.array([int(x) / 10 for x in pi_str])
pi

array([0.3, 0.1, 0.4, 0.1, 0.5, 0.9, 0.2, 0.6, 0.5, 0.3, 0.5, 0.8, 0.9,
       0.7, 0.9, 0.3, 0.2, 0.3, 0.8, 0.4, 0.6, 0.2, 0.6, 0.4, 0.3, 0.3,
       0.8, 0.3, 0.2, 0.7, 0.9, 0.5, 0. , 0.2, 0.8, 0.8, 0.4, 0.1, 0.9,
       0.7, 0.1, 0.6, 0.9, 0.3, 0.9, 0.9, 0.3, 0.7, 0.5, 0.1, 0. , 0.5,
       0.8, 0.2, 0. , 0.9, 0.7, 0.4, 0.9, 0.4, 0.4, 0.5, 0.9, 0.2, 0.3,
       0. , 0.7, 0.8, 0.1, 0.6, 0.4, 0. , 0.6, 0.2, 0.8, 0.6, 0.2, 0. ,
       0.8, 0.9, 0.9, 0.8, 0.6, 0.2, 0.8, 0. , 0.3, 0.4, 0.8, 0.2, 0.5,
       0.3, 0.4, 0.2, 0.1, 0.1, 0.7, 0. , 0.6, 0.7, 0.9, 0.8, 0.2, 0.1,
       0.4, 0.8, 0. , 0.8, 0.6, 0.5, 0.1, 0.3, 0.2, 0.8, 0.2, 0.3, 0. ,
       0.6, 0.6, 0.4, 0.7, 0. , 0.9, 0.3, 0.8, 0.4, 0.4, 0.6, 0. , 0.9,
       0.5, 0.5, 0. , 0.5, 0.8, 0.2, 0.2, 0.3, 0.1, 0.7, 0.2, 0.5, 0.3,
       0.5, 0.9, 0.4, 0. , 0.8, 0.1, 0.2, 0.8, 0.4, 0.8, 0.1, 0.1, 0.1,
       0.7, 0.4, 0.5, 0. , 0.2, 0.8, 0.4, 0.1, 0. , 0.2, 0.7, 0. , 0.1,
       0.9, 0.3, 0.8, 0.5, 0.2, 0.1, 0.1, 0. , 0.5, 0.5, 0.5, 0.

In [4]:
e_str = """2.7182818284590452353602874713526624977572470936999595749669676277240766303535
47594571382178525166427427466391932003059921817413596629043572900334295260595630
73813232862794349076323382988075319525101901157383418793070215408914993488416750
92447614606680822648001684774118537423454424371075390777449920695517027618386062
61331384583000752044933826560297606737113200709328709127443747047230696977209310
14169283681902551510865746377211125238978442505695369677078544996996794686445490
59879316368892300987931277361782154249992295763514822082698951936680331825288693
98496465105820939239829488793320362509443117301238197068416140397019837679320683
28237646480429531180232878250981945581530175671736133206981125099618188159304169
03515988885193458072738667385894228792284998920868058257492796104841984443634632
44968487560233624827041978623209002160990235304369941849146314093431738143640546
25315209618369088870701676839642437814059271456354906130310720851038375051011574
77041718986106873969655212671546889570350354021234078498193343210681701210056278
80235193033224745015853904730419957777093503660416997329725088687696640355570716
22684471625607988265178713419512466520103059212366771943252786753985589448969709
64097545918569563802363701621120477427228364896134225164450781824423529486363721
41740238893441247963574370263755294448337998016125492278509257782562092622648326
27793338656648162772516401910590049164499828931505660472580277863186415519565324
42586982946959308019152987211725563475463964479101459040905862984967912874068705
04895858671747985466775757320568128845920541334053922000113786300945560688166740
01698420558040336379537645203040243225661352783695117788386387443966253224985065
49958862342818997077332761717839280349465014345588970719425863987727547109629537
41521115136835062752602326484728703920764310059584116612054529703023647254929666
93811513732275364509888903136020572481765851180630364428123149655070475102544650
11727211555194866850800368532281831521960037356252794495158284188294787610852639
81395599006737648292244375287184624578036192981971399147564488262603903381441823
26251509748279877799643730899703888677822713836057729788241256119071766394650706
33045279546618550966661856647097113444740160704626215680717481877844371436988218
55967095910259686200235371858874856965220005031173439207321139080329363447972735
59552773490717837934216370120500545132638354400018632399149070547977805669785335
80489669062951194324730995876552368128590413832411607226029983305353708761389396
39177957454016137223618789365260538155841587186925538606164779834025435128439612
94603529133259427949043372990857315802909586313826832914771163963370924003168945
86360606458459251269946557248391865642097526850823075442545993769170419777800853
62730941710163434907696423722294352366125572508814779223151974778060569672538017
18077636034624592787784658506560507808442115296975218908740196609066518035165017
92504619501366585436632712549639908549144200014574760819302212066024330096412704
89439039717719518069908699860663658323227870937650226014929101151717763594460202
32493002804018677239102880978666056511832600436885088171572386698422422010249505
51881694803221002515426494639812873677658927688163598312477886520141174110913601
16499507662907794364600585194199856016264790761532103872755712699251827568798930
27617611461625493564959037980458381823233686120162437365698467037858533052758333
37939907521660692380533698879565137285593883499894707416181550125397064648171946
70834819721448889879067650379590366967249499254527903372963616265897603949857674
13973594410237443297093554779826296145914429364514286171585873397467918975712119
56187385783644758448423555581050025611492391518893099463428413936080383091662818
81150371528496705974162562823609216807515017772538740256425347087908913729172282
86115159156837252416307722544063378759310598267609442032619242853170187817729602
35413060672136046000389661093647095141417185777014180606443636815464440053316087
78314317444081194942297559931401188868331483280270655383300469329011574414756313
99972217038046170928945790962716622607407187499753592127560844147378233032703301
68237193648002173285734935947564334129943024850235732214597843282641421684878721
"""
e_str = e_str.replace('\n', '').replace('.', '')
print(len(e_str))
e = np.array([int(x) / 10 for x in e_str[:32*128]]).reshape(32, 128)
e

4157


array([[0.2, 0.7, 0.1, ..., 0.1, 0.3, 0.5],
       [0.9, 0.6, 0.6, ..., 0.2, 0.6, 0.4],
       [0.8, 0. , 0. , ..., 0.7, 0.2, 0.3],
       ...,
       [0.8, 0. , 0.7, ..., 0.3, 0.5, 0.4],
       [0.1, 0.3, 0. , ..., 0.7, 0. , 0.6],
       [0.5, 0.5, 0.3, ..., 0.3, 0.2, 0.8]])

In [5]:
model = Model()
model = model.to(device)

In [6]:
w1 = np.array([int(x) / 10 for x in pi_str[:128]], dtype=np.float32).reshape(1, -1).repeat(256, axis=0)
model.fc1.weight = nn.Parameter(torch.tensor(w1).to(device))

In [7]:
w2 = np.array([int(x) / 10 for x in pi_str[:256]], dtype=np.float32).reshape(1, -1).repeat(2**22, axis=0)
del model.fc2.weight
model.fc2.weight = nn.Parameter(torch.tensor(w2).to(device))
del w2

In [8]:
e_gpu = torch.tensor(e, dtype=torch.float32).to(device)

In [9]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)
model.train()

Model(
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=4194304, bias=True)
)

In [10]:
# 模型占显存 4696MB

In [11]:
optimizer.zero_grad()

i = 0
target = torch.range(i*32, (i+1)*32-1, dtype=torch.int64).to(device)
output = model(e_gpu)
loss += F.cross_entropy(output, target)


In [12]:
# 运行后占5724MB，运行需要 1028MB

In [13]:
loss.backward()

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524586445097/work/aten/src/THC/generic/THCStorage.cu:58

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

model.train()
with tqdm(range(2**17)) as pbar:
    for i in pbar:
        optimizer.zero_grad()
        
        loss = 0
        for j in range(i*32, (i+1)*32):
            target = torch.tensor((j,), dtype=torch.int64).to(device)
            output = model(e_gpu[j%32:j%32+1])
            loss += F.cross_entropy(output, target)
        
        loss.backward()
        optimizer.step()

In [ ]:
j